In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
import numpy as np
from sklearn.cluster import KMeans

from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from keras.applications import VGG16
from tensorflow.keras.utils import to_categorical


C:\Users\ASUS\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import cv2

data_path = r"Combined_images"
image_list = []
file_names = []

# Iterate over each file in the specified directory
for filename in os.listdir(data_path):
    if filename.endswith(('.jpg', '.png', '.tif', '.JPG')):
        file_path = os.path.join(data_path, filename)
        img = cv2.imread(file_path)
        # Append the image and its filename to the lists
        image_list.append(img)
        file_names.append(filename)

In [3]:
# Function to extract the number from the image filename
def extract_number_from_filename(filename):
    # Assuming the number is after 'combined_' and before the first '.'
    return filename.split('_')[1].split('.')[0]

In [4]:
clean_file_names = []
for name in file_names:
    clean_file_names.append(extract_number_from_filename(name))

In [5]:
import csv

# Sample data (replace these with your actual data)
images = image_list
image_names = clean_file_names

# Function to read the CSV and create a dictionary of image names to labels
def read_csv_to_dict(csv_file):
    df = pd.read_csv(csv_file)

    # Extract image names and labels
    image_names = df.iloc[:, 0].values
    labels = df.iloc[:, -1].values

    # Create a list of image-label pairs
    image_label_array = np.array([[name, label] for name, label in zip(image_names, labels)])
    
    # Convert the NumPy array to a dictionary
    image_label_dict = {row[0]: row[1] for row in image_label_array}
    return image_label_dict


# Function to create the list of images and their corresponding labels
def create_image_label_list(images, image_names, image_label_dict):
    image_label_list = []
    for image, name in zip(images, image_names):
        if name in image_label_dict:
            label = image_label_dict[name]
            image_label_list.append([name,image, label])
    return image_label_list

# Main execution
csv_file_path = 'image_labels.csv'  # Replace with your actual CSV file path
image_label_dict = read_csv_to_dict(csv_file_path)
image_label_list = create_image_label_list(images, image_names, image_label_dict)

# Output the result
for item in image_label_list:
    print(item)

['150007', array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8), 'al-Mansuriyah']
['150008', array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 

        [4, 6, 6]]], dtype=uint8), 'al-Mansuriyah']
['150890', array([[[7, 9, 9],
        [6, 8, 8],
        [6, 8, 8],
        ...,
        [6, 8, 8],
        [6, 8, 8],
        [6, 8, 8]],

       [[7, 9, 9],
        [6, 8, 8],
        [5, 7, 7],
        ...,
        [6, 8, 8],
        [6, 8, 8],
        [6, 8, 8]],

       [[6, 8, 8],
        [5, 7, 7],
        [5, 7, 7],
        ...,
        [6, 8, 8],
        [6, 8, 8],
        [6, 8, 8]],

       ...,

       [[3, 5, 5],
        [3, 5, 5],
        [3, 5, 5],
        ...,
        [4, 6, 6],
        [3, 5, 5],
        [3, 5, 5]],

       [[3, 5, 5],
        [3, 5, 5],
        [3, 5, 5],
        ...,
        [4, 6, 6],
        [3, 5, 5],
        [3, 5, 5]],

       [[3, 5, 5],
        [3, 5, 5],
        [3, 5, 5],
        ...,
        [4, 6, 6],
        [3, 5, 5],
        [3, 5, 5]]], dtype=uint8), 'al-Mahdiyah']
['150891', array([[[3, 5, 5],
        [3, 5, 5],
        [3, 5, 5],
        ...,
        [4, 6, 6],
        [4, 6, 6],
  

        [ 4,  9,  8]]], dtype=uint8), 'al-Mansuriyah']
['151159', array([[[ 4,  9,  8],
        [ 3,  8,  7],
        [ 3,  8,  7],
        ...,
        [24, 29, 28],
        [20, 25, 24],
        [25, 30, 29]],

       [[ 4,  9,  8],
        [ 3,  8,  7],
        [ 3,  8,  7],
        ...,
        [24, 29, 28],
        [22, 27, 26],
        [26, 31, 30]],

       [[ 4,  9,  8],
        [ 3,  8,  7],
        [ 3,  8,  7],
        ...,
        [21, 26, 25],
        [21, 26, 25],
        [25, 30, 29]],

       ...,

       [[ 4,  7,  5],
        [ 4,  7,  5],
        [ 4,  7,  5],
        ...,
        [ 3,  8,  7],
        [ 3,  8,  7],
        [ 3,  8,  7]],

       [[ 4,  7,  5],
        [ 4,  7,  5],
        [ 4,  7,  5],
        ...,
        [ 3,  8,  7],
        [ 3,  8,  7],
        [ 3,  8,  7]],

       [[ 4,  7,  5],
        [ 4,  7,  5],
        [ 4,  7,  5],
        ...,
        [ 3,  8,  7],
        [ 3,  8,  7],
        [ 3,  8,  7]]], dtype=uint8), 'al-Mahdiyah']
['151160',

In [6]:
len(image_label_list)

625

In [7]:
df = pd.read_csv('image_labels.csv', names=['image_num','b','c','label'])

In [8]:
df['label'].value_counts()

Misr                        251
al-Mansuriyah               223
m.                          104
al-Mahdiyah                  40
(al-Mansuriyah)              10
Filastin                      9
al-Mansuriyah?                8
uncertain                     5
(Atarablus)                   4
al-Qahirah al-Mahrusah        4
Sur                           3
Barqa                         3
Atarablus                     3
Makka                         2
Tabariyyah                    2
Zawilah                       2
(al-Qahirah al-Mahrusah)      1
Dimashq                       1
Madinat Rasul Allah?          1
no mint                       1
Name: label, dtype: int64

In [9]:
two_largest_list = []
for instance in image_label_list:
    if instance[2]=='Misr' or instance[2]=='al-Mansuriyah'or instance[2]=='(al-Mansuriyah)':
        if instance[2]=='(al-Mansuriyah)':
            instance[2]='al-Mansuriyah'
            print(instance[2])
        two_largest_list.append(instance)


al-Mansuriyah
al-Mansuriyah
al-Mansuriyah
al-Mansuriyah
al-Mansuriyah
al-Mansuriyah
al-Mansuriyah
al-Mansuriyah
al-Mansuriyah
al-Mansuriyah


In [10]:
len(two_largest_list)

452

In [63]:
# Convert resized_image_list to numpy array
X = np.array([row[1] for row in two_largest_list])
Y = np.array([row[2] for row in two_largest_list])

In [64]:

X = X/255.0
Y = np.array([0 if x == 'Misr' else 1 for x in Y])
Y_1 = to_categorical(Y, num_classes=2)

In [65]:
Y_1

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.

In [14]:
X.shape

(452, 300, 600, 3)

In [68]:
# Split the dataset into training and testing sets (80% train, 20% test)
X_train_first, X_test, y_train_first, y_test = train_test_split(X,Y_1, test_size=0.2, random_state=42)

In [69]:
X_train, X_val, y_train, y_val = train_test_split(X_train_first,y_train_first, test_size=0.2, random_state=42)

In [70]:
X_train.shape

(288, 300, 600, 3)

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Flatten the input data for XGBoost
X_flat = X.reshape(X.shape[0], -1)

# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_flat, Y, test_size=0.2, random_state=42)


# Create and train the XGBoost model
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred = xgb_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'XGBoost Accuracy: {accuracy}')

In [17]:
import os
import glob
import cv2
from keras.utils import to_categorical
import pandas as pd
import numpy as np
import tensorflow as tf
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn import metrics
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall ,F1Score

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
from numpy import asarray

In [71]:
# model = Sequential()
# model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
#                  activation ='relu', input_shape = (300,600,3)))
# model.add(MaxPool2D(pool_size=(2,2)))
# model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
#                  activation ='relu'))
# model.add(MaxPool2D(pool_size=(2,2)))
# model.add(Flatten())
# model.add(Dense(512, activation = "relu"))
# model.add(Dropout(0.2))
# model.add(Dense(256, activation = "relu"))
# model.add(Dropout(0.2))
# model.add(Dense(64, activation = "relu"))
# model.add(Dense(2, activation = "softmax"))
model = Sequential()

# First convolutional layer with fewer filters
model.add(Conv2D(filters=16, kernel_size=(3, 3), padding='Same', 
                 activation='relu', input_shape=(300, 600, 3)))
model.add(MaxPool2D(pool_size=(2, 2)))

# Second convolutional layer with fewer filters
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='Same', 
                 activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

# Flattening the feature maps
model.add(Flatten())

# Fully connected layer with fewer neurons
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

# Output layer
model.add(Dense(2, activation='softmax'))

In [72]:
model.compile(optimizer = 'adam', loss= "categorical_crossentropy", metrics=["accuracy"])
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 300, 600, 16)      448       
                                                                 
 max_pooling2d_20 (MaxPooli  (None, 150, 300, 16)      0         
 ng2D)                                                           
                                                                 
 conv2d_21 (Conv2D)          (None, 150, 300, 32)      4640      
                                                                 
 max_pooling2d_21 (MaxPooli  (None, 75, 150, 32)       0         
 ng2D)                                                           
                                                                 
 flatten_10 (Flatten)        (None, 360000)            0         
                                                                 
 dense_20 (Dense)            (None, 128)             

In [20]:
early_stopping = EarlyStopping(monitor='val_loss', patience=1, verbose=1)

In [73]:
history = model.fit(
    X_train, y_train,
    epochs=7,
    batch_size=32,
    validation_data=(X_val, y_val)
#     callbacks=[early_stopping]
)

# Evaluate the model on the validation set
# val_loss, val_accuracy = model.evaluate(X_test, y_test)

Epoch 1/7
9/9 [==============================] - 13s 1s/step - loss: 14.0767 - accuracy: 0.4410 - val_loss: 5.0549 - val_accuracy: 0.4795
Epoch 2/7
9/9 [==============================] - 10s 1s/step - loss: 2.7295 - accuracy: 0.5139 - val_loss: 0.6151 - val_accuracy: 0.6027
Epoch 3/7
9/9 [==============================] - 9s 1s/step - loss: 0.6304 - accuracy: 0.7153 - val_loss: 0.4220 - val_accuracy: 0.8082
Epoch 4/7
9/9 [==============================] - 9s 1s/step - loss: 0.2588 - accuracy: 0.9132 - val_loss: 0.3600 - val_accuracy: 0.8356
Epoch 5/7
9/9 [==============================] - 9s 993ms/step - loss: 0.1490 - accuracy: 0.9583 - val_loss: 0.2037 - val_accuracy: 0.9041
Epoch 6/7
9/9 [==============================] - 8s 936ms/step - loss: 0.0679 - accuracy: 0.9861 - val_loss: 0.1896 - val_accuracy: 0.9178
Epoch 7/7
9/9 [==============================] - 8s 908ms/step - loss: 0.0296 - accuracy: 1.0000 - val_loss: 0.1535 - val_accuracy: 0.9315


In [74]:
loss,accuracy = model.evaluate(X_test,y_test, verbose=0)
print(f"Loss: {loss} Accuracy: {accuracy}")

Loss: 0.13547241687774658 Accuracy: 0.9560439586639404


al-Mahdiyah = 37 , m. = 91 , Misr = 235 , al-Mansuriyah = 217

In [23]:
three_largest_list = []
for instance in image_label_list:
    if instance[2]=='Misr' or instance[2]=='al-Mansuriyah' or instance[2]=='(al-Mansuriyah)' or instance[2]=='m.':
        if instance[2]=='(al-Mansuriyah)':
            instance[2]='al-Mansuriyah'
            print(instance[2])
        three_largest_list.append(instance)


In [24]:
len(three_largest_list)

543

In [38]:
# Convert resized_image_list to numpy array
X = np.array([row[1] for row in three_largest_list])
Y = [row[2] for row in three_largest_list]


In [39]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                    shear_range = 0.2,
                                    zoom_range = 0.1,
                                    rotation_range = 25,
                                    width_shift_range = 0.1,
                                    height_shift_range = 0.1,
                                    horizontal_flip = True,
                                    validation_split = 0.2)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [26]:
X = X/255.0
Y = np.array([0 if x == 'Misr' else 1 if x == 'al-Mansuriyah' else 2 for x in Y])
Y_2 = to_categorical(Y, num_classes=3)

In [27]:
Y_2

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [57]:
# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X,Y_2, test_size=0.2, random_state=42)

In [58]:
X_train_2, X_val, y_train_2, y_val = train_test_split(X_train,y_train, test_size=0.2, random_state=42)

In [29]:
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [59]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


model = Sequential()

# First convolutional layer with fewer filters
model.add(Conv2D(filters=16, kernel_size=(3, 3), padding='Same', 
                 activation='relu', input_shape=(300, 600, 3)))
model.add(MaxPool2D(pool_size=(2, 2)))

# Second convolutional layer with fewer filters
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='Same', 
                 activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

# Flattening the feature maps
model.add(Flatten())

# Fully connected layer with fewer neurons
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

# Output layer for 3 classes
model.add(Dense(3, activation='softmax'))
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()




Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 300, 600, 16)      448       
                                                                 
 max_pooling2d_18 (MaxPooli  (None, 150, 300, 16)      0         
 ng2D)                                                           
                                                                 
 conv2d_19 (Conv2D)          (None, 150, 300, 32)      4640      
                                                                 
 max_pooling2d_19 (MaxPooli  (None, 75, 150, 32)       0         
 ng2D)                                                           
                                                                 
 flatten_9 (Flatten)         (None, 360000)            0         
                                                                 
 dense_18 (Dense)            (None, 128)              

In [60]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [61]:
# Train the model
history = model.fit(
     X_train_2, y_train_2,
#     datagen.flow(X_train, y_train, batch_size=32),
    epochs=8,  # Increase the number of epochs to allow for more training
    validation_data=(X_val, y_val),
    callbacks=[early_stopping]
)


Epoch 1/8
11/11 [==============================] - 14s 1s/step - loss: 3360.4734 - accuracy: 0.3545 - val_loss: 437.1253 - val_accuracy: 0.4483
Epoch 2/8
11/11 [==============================] - 12s 1s/step - loss: 137.4038 - accuracy: 0.4063 - val_loss: 30.0688 - val_accuracy: 0.4483
Epoch 3/8
11/11 [==============================] - 12s 1s/step - loss: 14.2860 - accuracy: 0.5418 - val_loss: 5.5728 - val_accuracy: 0.4598
Epoch 4/8
11/11 [==============================] - 11s 1s/step - loss: 2.4727 - accuracy: 0.6254 - val_loss: 1.7926 - val_accuracy: 0.5172
Epoch 5/8
11/11 [==============================] - 10s 921ms/step - loss: 0.7466 - accuracy: 0.7464 - val_loss: 1.2698 - val_accuracy: 0.4713
Epoch 6/8
11/11 [==============================] - 11s 975ms/step - loss: 0.4810 - accuracy: 0.8098 - val_loss: 1.1169 - val_accuracy: 0.5747
Epoch 7/8
11/11 [==============================] - 10s 905ms/step - loss: 0.4049 - accuracy: 0.8501 - val_loss: 1.0942 - val_accuracy: 0.5287
Epoch 8/8

In [62]:
loss,accuracy = model.evaluate(X_test,y_test, verbose=0)
print(f"Loss: {loss} Accuracy: {accuracy}")

Loss: 1.4520976543426514 Accuracy: 0.47706422209739685


In [47]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical

# Assuming X_train, y_train, X_test, y_test are your data arrays
# Ensure y_train and y_test are one-hot encoded
if len(y_train.shape) == 1:
    y_train = to_categorical(y_train, num_classes=3)
if len(y_test.shape) == 1:
    y_test = to_categorical(y_test, num_classes=3)

# Data Augmentation
train_datagen = ImageDataGenerator(
    rotation_range=25,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    shear_range = 0.2,
    validation_split=0.2  # Using validation split
)

test_datagen = ImageDataGenerator()

# Train and Validation Sets
training_set = train_datagen.flow(
    X_train, y_train,
    batch_size=32,
    shuffle=True,
    seed=2,
    subset='training'
)

validation_set = train_datagen.flow(
    X_train, y_train,
    batch_size=32,
    shuffle=True,
    seed=2,
    subset='validation'
)

# Test Set
test_set = test_datagen.flow(
    X_test, y_test,
    batch_size=32,
    shuffle=False
)

# Check the shapes of the batches
print(next(training_set)[0].shape)  # Should be (32, 300, 600, 3)
print(next(validation_set)[0].shape)  # Should be (32, 300, 600, 3)

# Model definition
model = Sequential()

# First convolutional layer
model.add(Conv2D(filters=16, kernel_size=(3, 3), padding='Same', activation='relu', input_shape=(300, 600, 3)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

# Second convolutional layer
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='Same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

# Flattening the feature maps
model.add(Flatten())

# Fully connected layer
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Output layer for 3 classes
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

# Train the model with data augmentation
history = model.fit(
    training_set,
    epochs=50,  # Increase the number of epochs to allow for more training
    validation_data=validation_set,
    callbacks=[early_stopping, reduce_lr]
)


(32, 300, 600, 3)
(32, 300, 600, 3)
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 300, 600, 16)      448       
                                                                 
 batch_normalization_6 (Bat  (None, 300, 600, 16)      64        
 chNormalization)                                                
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 150, 300, 16)      0         
 ng2D)                                                           
                                                                 
 conv2d_13 (Conv2D)          (None, 150, 300, 32)      4640      
                                                                 
 batch_normalization_7 (Bat  (None, 150, 300, 32)      128       
 chNormalization)                                                
                  

In [48]:
loss,accuracy = model.evaluate(X_test,y_test, verbose=0)
print(f"Loss: {loss} Accuracy: {accuracy}")

Loss: 1.3274916410446167 Accuracy: 0.4128440320491791
